In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
from transformers import AdamW
from transformers import AutoTokenizer, AutoModel

In [4]:
import os
import csv
import pandas as pd
from tqdm import tqdm
import torch 
from torch import nn

from sklearn.model_selection import train_test_split
import torch 
#import datasets
#from datasets import Dataset

from tqdm.notebook import tqdm_notebook
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
# import hazm
# import patoolib
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
! wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

--2023-05-15 18:40:59--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2023-05-15 18:40:59--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  48.2MB/s    in 1.6s    

2023-05-15 18:41:01 (48.2 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]



In [8]:
! unzip trainingandtestdata.zip

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [9]:
train_file_name = 'training.1600000.processed.noemoticon.csv'

In [10]:
test_file_name = 'testdata.manual.2009.06.14.csv'

In [11]:
# 0 = negative, 2 = neutral, 4 = positive

In [12]:
colnames=['polarity', 'id', 'date', 'query','user','text'] 
df_train = pd.read_csv(train_file_name , encoding = "ISO-8859-1", names=colnames, header=None)
df_test = pd.read_csv(test_file_name , encoding = "ISO-8859-1", names=colnames, header=None)

In [13]:
frames = [df_train, df_test]
df = pd.concat(frames)

In [14]:
df_test

,polarity,id,date,query,user,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
...,...,...,...,...,...,...
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [15]:
df

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [16]:
dataset = df[['polarity','text']]

In [17]:
dataset['polarity'][dataset['polarity']== 4 ] = 1

<ipython-input-17-9306a42d4560>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['polarity'][dataset['polarity']== 4 ] = 1
<ipython-input-17-9306a42d4560>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['polarity'][dataset['polarity']== 4 ] = 1


In [18]:
dataset

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
493,2,Ask Programming: LaTeX or InDesign?: submitted...
494,0,"On that note, I hate Word. I hate Pages. I hat..."
495,1,Ahhh... back in a *real* text editing environm...
496,0,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [19]:
# lowering text
dataset['text'] = dataset['text'].apply(lambda x: x.lower())
dataset.head()

<ipython-input-19-563faafa41de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x: x.lower())


,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [20]:
import string
import re
string.punctuation

#defining the function to remove punctuation
def remove_punctuation(text):
    text = re.sub('\n', ' ', text)
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#storing the puntuation free text
dataset['text'] = dataset['text'].apply(lambda x:remove_punctuation(x))

<ipython-input-20-dd23128cc284>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x:remove_punctuation(x))


In [21]:
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= " ".join([i for i in text.split() if i not in stopwords])
    return output

#applying the function
dataset['text'] = dataset['text'].apply(lambda x:remove_stopwords(x))

<ipython-input-21-38ae90e16a0e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x:remove_stopwords(x))


In [22]:
def cleaning_repeating_char(text):
    return re.sub(r'(\w)\1{2,}', r'\1',text)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_repeating_char(x))

<ipython-input-22-9f853cb9a5d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x: cleaning_repeating_char(x))


In [23]:
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_URLs(x))

<ipython-input-23-2e423e1fb105>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x: cleaning_URLs(x))


In [24]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))


<ipython-input-24-7083fe4d08c3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))


In [25]:
dataset

,polarity,text
0,0,switchfoot httptwitpiccomyzl aw thats bummer s...
1,0,upset cant update facebook texting might cry r...
2,0,kenichan dived many times ball managed save r...
3,0,whole body feels itchy like fire
4,0,nationwideclass behaving im mad cant see
...,...,...
493,2,ask programming latex indesign submitted calci...
494,0,note hate word hate pages hate latex said hate...
495,1,ah back real text editing environment lt latex
496,0,trouble iran see hmm iran iran far away flocko...


In [26]:
# tokenization
#defining function for tokenization
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

def tokenization(text):
    return tk.tokenize(text)

#applying function to the column
dataset['text'] = dataset['text'].apply(lambda x: tokenization(x))


<ipython-input-26-ee6dbb3b2b7b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x: tokenization(x))


In [27]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  
  return lemm_text
  
dataset['text'] = dataset['text'].apply(lambda x:lemmatizer(x))

<ipython-input-27-5f3e5f366113>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = dataset['text'].apply(lambda x:lemmatizer(x))


In [28]:
dataset

,polarity,text
0,0,"[switchfoot, httptwitpiccomyzl, aw, thats, bum..."
1,0,"[upset, cant, update, facebook, texting, might..."
2,0,"[kenichan, dived, many, time, ball, managed, s..."
3,0,"[whole, body, feel, itchy, like, fire]"
4,0,"[nationwideclass, behaving, im, mad, cant, see]"
...,...,...
493,2,"[ask, programming, latex, indesign, submitted,..."
494,0,"[note, hate, word, hate, page, hate, latex, sa..."
495,1,"[ah, back, real, text, editing, environment, l..."
496,0,"[trouble, iran, see, hmm, iran, iran, far, awa..."


In [29]:
!pip install datasets
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.7 MB/s eta 0:00:00


In [30]:
# val_dataa = dataset[10000:12000]
train_dataa = dataset[:1600000]
test_dataa = dataset[1600000:]

In [31]:
train_data = Dataset.from_pandas(train_dataa)
# val_data = Dataset.from_pandas(val_dataa)
test_data = Dataset.from_pandas(test_dataa)

# Glove

In [32]:
# !wget https://nlp.stanford.edu/data/glove.6B.zip

In [33]:
# !unzip glove*.zip

In [34]:
vocab,embeddings = [],[]
with open('/content/glove.6B.100d.txt','rt',encoding="utf8") as fi:
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    i_word = full_content[i].split(' ')[0]
    i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]

    if len(i_embeddings) == 100:
      # continue
      vocab.append(i_word)
      embeddings.append(i_embeddings)

In [35]:
vocab_npa = np.array(vocab)
embs_npa = np.array(embeddings)

In [36]:
vocab_npa = np.insert(vocab_npa, 0, '<pad>')
vocab_npa = np.insert(vocab_npa, 1, '<unk>')
print(vocab_npa[:20])

pad_emb_npa = np.zeros((1,embs_npa.shape[1]))   #embedding for '<pad>' token.
unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True)    #embedding for '<unk>' token.

#insert embeddings for pad and unk tokens at top of embs_npa.
embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,embs_npa))
print(embs_npa.shape)

['<pad>' '<unk>' 'the' ',' '.' 'of' 'to' 'and' 'in' 'a' '"' "'s" 'for' '-'
 'that' 'on' 'is' 'was' 'said' 'with']
(26892, 100)


In [37]:
word_to_index = dict(zip(list(vocab_npa),range(len(vocab_npa))))

In [38]:
def get_ids(txt):
  ids = []
  for token in txt:
    id = word_to_index.get(token,-1)
    if id != -1:
      ids.append(id)
    else:
      ids.append(word_to_index["<unk>"])
  return ids

In [39]:
def preprocess(record):
  max_len = 512 
  comment = record["text"]
  label = record["polarity"]

  comment_ids = get_ids(comment)

  return { "input_ids" : comment_ids[:max_len]  , "label": label}

In [40]:
train_dataset = train_data.map(preprocess ,remove_columns = train_data.column_names)
test_dataset = test_data.map(preprocess,remove_columns = test_data.column_names)
# val_dataset = val_data.map(preprocess,remove_columns = val_data.column_names)

Map:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [41]:
train_dataset.set_format(type='torch')
test_dataset.set_format(type='torch')
# val_dataset.set_format(type='torch')

In [42]:
def collate_batch(batch):
  input_ids =[]
  labels = []
  for b in batch:
    input_ids.append(b["input_ids"])
    labels.append(b["label"])
  input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=0.0)
  return {"input_ids":input_ids ,"label":torch.Tensor(labels)}

In [43]:
def validation( model , val_loader , loss_fn , device = "cuda"  ):
  val_loss = 0.0
  model.eval()
  reals = []
  preds_list = []
  for batch in val_loader:
    input_ids = batch["input_ids"].to(device)
    labels = batch["label"].to(device)
    with torch.no_grad():
      preds = model(input_ids = input_ids )
    loss = loss_fn(preds , labels.to(torch.long))
    val_loss += loss.item()
    reals += labels.cpu().numpy().tolist()
    preds_list += preds.argmax(axis=1).cpu().numpy().tolist()

  val_loss = val_loss / len(val_loader)
  accuracy = accuracy_score(reals, preds_list, normalize=True)
  print(classification_report(reals , preds_list ))
  return val_loss ,accuracy

In [44]:
def train(model , optimizer , loss_fn , train_loader  , test_loader , epochs , device):
  
  model.to(device)
  for epoch in range(epochs):
    training_loss = 0.0
    model.train()
    t_labels = []
    t_preds = []
    for batch in tqdm_notebook(train_loader):
      optimizer.zero_grad()
      input_ids = batch["input_ids"].to(device)
      labels = batch["label"].to(device)
      preds = model(input_ids)
      # print("preds :", preds)
      # print("labels :", labels)
      loss = loss_fn(preds , labels.to(torch.long))
      t_labels += labels.cpu().numpy().tolist()
      loss.backward()
      optimizer.step()
      training_loss += loss.item()
      t_preds +=  preds.argmax(axis=1).cpu().numpy().tolist()
    training_loss = training_loss / len(train_loader)
    train_accuracy =  accuracy_score(t_labels, t_preds, normalize=True)
    print('Epoch: {} ,training loss: {:.2f} , train accuracy: {:.2f} '.format(epoch , training_loss ,train_accuracy))
    test_loss , test_accuracy = validation(model , test_loader , loss_fn , device  )
    print('test loss: {:.2f}  test accuracy: {:.2f}'.format( test_loss , test_accuracy))

# lstm

In [53]:
class LSTM(nn.Module):
  def __init__(self , embs_npa):
    super(LSTM,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float())
    self.lstm = nn.LSTM(100,64,num_layers=1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.act_func = nn.ReLU()
    self.fc = nn.Linear(64, 3)
    

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    _ , (last_hidden_layer , _) = self.lstm(embeddings)
    out = self.dropout(self.act_func(self.fc(last_hidden_layer[-1])))
    return out

In [54]:
from torch.optim import Adam
epoch = 5
batch_size=16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTM(embs_npa)
optimizer = Adam(model.parameters(), lr=2e-5) 
loss_fn = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(train_dataset , batch_size, collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size , drop_last=True , collate_fn=collate_batch )
train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.27 , train accuracy: 0.89 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 9.12  test accuracy: 0.37


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch: 1 ,training loss: 0.27 , train accuracy: 0.88 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 9.78  test accuracy: 0.37


...

Epoch: 3 ,training loss: 0.25 , train accuracy: 0.89 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 9.94  test accuracy: 0.37


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.24 , train accuracy: 0.90 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 9.94  test accuracy: 0.37


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# gru

In [55]:
class GRU(nn.Module):
  def __init__(self , embs_npa):
    super(GRU,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float())
    self.gru = nn.GRU(100,64,num_layers=1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.fc = nn.Linear(64, 3)
    self.act_func = nn.ReLU()

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    _ , last_hidden_layer = self.gru(embeddings)
    out = self.dropout(self.act_func(self.fc(last_hidden_layer[-1])))
    return out

In [56]:
epoch = 5
batch_size=16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRU(embs_npa)
optimizer = Adam(model.parameters(), lr=2e-5) 
loss_fn = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(train_dataset , batch_size, collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size , drop_last=True , collate_fn=collate_batch )
train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.27 , train accuracy: 0.90 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 8.94  test accuracy: 0.37


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch: 1 ,training loss: 0.25 , train accuracy: 0.90 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 9.73  test accuracy: 0.37


...

Epoch: 3 ,training loss: 0.24 , train accuracy: 0.90 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 10.39  test accuracy: 0.37


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.24 , train accuracy: 0.90 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       175
         1.0       0.37      1.00      0.54       182
         2.0       0.00      0.00      0.00       139

    accuracy                           0.37       496
   macro avg       0.12      0.33      0.18       496
weighted avg       0.13      0.37      0.20       496

test loss: 10.44  test accuracy: 0.37


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
